# Chiral solutions for Type-II Dirac seesaw with a Dark Symmetry

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

## Load full solutions

In [2]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(148097, 5)

Check that solutions are unique

In [3]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(148097, 6)

In [4]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [5]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

95358

## Check phenomenological conditions
$$ \nu+m+s=0$$

In [6]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [7]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    triplet=False
    for x in ll:
        if ll.count(x)>1:
            if ll.count(x)>2:
                triplet=True
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2 and triplet:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (.ν,ν,...)
    '''
    fs=[]
    mrp=[]
    νrp=[]
    for x in ll:
        if ll.count(x)==1:
            fs.append(x)
        elif ll.count(x)==3:
            mrp.append(x)
        elif ll.count(x)>3:
            mrp.append(x)
            fs.append(x)
        elif ll.count(x)==2:
            νrp.append(x)
    mrp=sorted(np.unique(mrp))
    νrp=sorted(np.unique(νrp))
    fs=sorted(np.unique(fs))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for m in mrp:
        for ν in νrp:
            #if (mrp[0]+ν in [sum(k)  for k  in itertools.combinations(fs,2)]+list(2*np.array(fs))):
            s=-m-ν# ,fs,[sum(k)  for k  in itertools.combinations(fs,2)],list(2*np.array(fs)))
            nom=mrp.copy()
            kk=nom.remove(m)
            noν=νrp.copy()
            kk=noν.remove(ν)
            massless=fs+nom+noν
            massless=extract_Dirac_and_Majorana(massless,s)
            slt={'[m,ν,s]':[m,ν,s],'massless':massless}
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
            sltn.append(slt)
    return sltn

#assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
#assert best_higgs_singlet([1,-11,9])==(2, [])
#assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
#assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

In [8]:
max([1,2,5])

5

In [9]:
def rank(l,massless,s):
    '''
    Return 0 if the integers in massless appears as s-paired same-dimension multiplets in l, 1 otherwise.
    s-paired means that exist two intengers in massless such that n1+n2=±s. (n1 can be equal to n2)
    
    For example, for l containing only the massless=[1,2] integers:
        l=[1,2]; s=3 → 0: 3-paired multiplet: ([1],[2]) of 1-dimension multiplets (A.K.A singlet)
        l=[1,1,2]; s=3 → 1: unpaired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2]; s=3 → 0: 3-paired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2,2]; s=3 → 1: unpaired multiplets: ([1,1,1],[2,2]) doblet and singlet
    massless=[1,2,3]
    l=[1,2,3]; s=4 → 0: 4-paired multiplet: ([1],[3]) of singlets
                            4-paired multiplet: ([1],[3]) of singlet
    '''
    #Keep massless
    l=[x for x in  l if x in massless ]
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    #Get s-multiplets
    xxx=[l.count(x) for x in xx if l.count(x)!=1] 
    if len(xxx)%2==0: # s-multiplets must be paired
        # Check that each mutiplet is paired
        if 1 in [xxx.count(x)%2 for x in set(xxx)]:
            return 1
        else:
            #list of only 0's → if dimension of multiplets are paired
            return 0 # 
    else:
        return 1

def remove_majorana(l,s):
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    return xx

def check_rank(l,ν):
    massless=list(set([x for x in  l if x!=ν ]))
    s=-ν
    return rank(l,massless,s)

def extract_massless(l,ν):
    '''
    Return 0 if the integers in massless appears as s-paired same-dimension multiplets in l, 1 otherwise.
    s-paired means that exist two intengers in massless such that n1+n2=±s. (n1 can be equal to n2)
    
    For example, for l containing only the massless=[1,2] integers:
        l=[1,2]; s=3 → 0: 3-paired multiplet: ([1],[2]) of 1-dimension multiplets (A.K.A singlet)
        l=[1,1,2]; s=3 → 1: unpaired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2]; s=3 → 0: 3-paired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2,2]; s=3 → 1: unpaired multiplets: ([1,1,1],[2,2]) doblet and singlet
    massless=[1,2,3]
    l=[1,2,3]; s=4 → 0: 4-paired multiplet: ([1],[3]) of singlets
                            4-paired multiplet: ([1],[3]) of singlet
    '''
    s=-ν
    #Keep massless
    m=[x for x in  l if x!=ν ]
    #m=[x for x in  m if m.count(x)!=1 ]
    mltps=list( set( [m.count(x) for x in m ]  ))
    #print(mltps)
    Maj=[]
    if len(mltps)>1:
        for x in set(m):
            for z in [y for y in set(m) if y!=x and abs(y+x)==abs(s) and abs(x)<abs(y) ]:
                if l.count(x)>l.count(z):
                    Maj.append(x)
                elif l.count(x)<l.count(z):
                    Maj.append(z)
    if len(Maj)==1:
        return Maj[0]*2,None
    elif len(Maj)>1:
        return Maj[0]*2,Maj[1:] 
    else:
        return None,None

    
def extract_sp_massless(l,massless,sp):
    #Restore full massless list
    m=[x for x in  l if x in massless ]
    #Obtain the multiplets
    mltps=list( set( [m.count(x) for x in m]  ))
    Maj=[]
    if len(mltps)>1:
        for x in set(m):
            for z in [y for y in set(m) if y!=x and abs(y+x)==sp and abs(x)<abs(y) ]:
                if l.count(x)>l.count(z):
                    Maj.append(x)
                elif l.count(x)<l.count(z):
                    Maj.append(z)
    return Maj
    


#******* rank        ****************
assert rank([1,2],[1,2],3)==0
assert rank([1,1,2,2,2],[1,2],3)==1
assert rank([1,2,3],[1,2,3],4)==0
assert rank([1, 1, -4, 9, -10, -10, 16, 16, -19],[9, -19, 1],18)==1
assert rank( [1, -4, -4, 9, 9, -11], [-11],22)==0
assert rank( [-16,-16,-16, 17, 17],[-16, 17],1)==1
assert rank( [-16,-16,-16, 17, 17,-18,-18,19,19,19],[-16, 17,-18,19],1)==0
#************************************
#********* extract_massless ************
assert extract_massless([1,-5, -6, 10, 10,10, 9,9,2,2,-11, -11],-11)==(20, None)
assert extract_massless([1,1,-5, -6, 10, 10,10, 9,9,2,2,-11, -11],-11)==(20, None)
assert extract_massless([1,1,-5, -6, 10, 10,10, 9,9,2,2,2,-11, -11],-11)==(20, [2])
assert extract_massless([1, 1, 1, -4, -4, 5],1)==(-8, None)
#******************************************
#********* extract_sp_massless ************
assert extract_sp_massless([1, 1, 1, 4, 4, 5, -10, -10, -10, 14],[1,5],6)==[1]
#******************************************

In [38]:
def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (ν,ν,...)
    '''
    fs=[]
    νrp=[]
    for x in np.unique(ll):
        if ll.count(x)==2 or ll.count(x)==3:
            νrp.append(x)
    νrp=sorted(np.unique(νrp))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for ν in νrp:
        s=-ν
        slt={'[ν,s]':[ν,s]}
        fs=list(np.unique([x for x in ll if x!=ν]))
        massless=extract_Dirac_and_Majorana(fs,s)
        if len(massless)>0:
            slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            if rank(ll,massless,slt['sp'])==1:
                slt['sp_massless']=slt['sp_massless']+extract_sp_massless(  ll,massless,slt['sp']  )
            if check_rank(ll,ν)==1:
                spextra,tmp=extract_massless(ll,ν)
                if spextra:
                    slt['sp_massless']=slt['sp_massless']+[spextra//2]
                

        else: # Not extra sp required
            if check_rank(ll,ν)==1:
                slt['sp'],slt['sp_massless']=extract_massless(ll,ν)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
        sltn.append(slt)
    #Extract best solution
    sltn0=[d for d in sltn if d.get('sp') is None]
    if len(sltn0)>0: #Solution without extra `sp`
        return sltn0[0]
    sltn0=[d for d in sltn if d.get('sp') and isinstance(d.get('sp_massless'),list) and len(d.get('sp_massless'))==0]
    if len(sltn0)>0: #Solution with extra `sp` empty sp_massless
        return sltn0[0]
    ##TODO: Add sp  but with None sp_massless
    sltn0=[d for d in sltn if d.get('sp') and d.get('sp_massless') is None]
    if len(sltn0)>0: #Solution with extra `sp` empty sp_massless
        return sltn0[0]
    return np.nan

assert tree_level_ii([1,1,-4,-5,9,9,9,-10,-10]).get('[ν,s]')==[9, -9]
assert tree_level_ii([1, 2, 2, -3, -3, -3, 4]) is np.nan #ν→ -3 [1,2,2]→ massless 2

### Initialize filtered solutions

In [39]:
cl=ds.copy()

Find solutions that satisfy the condition

In [40]:
cl['tree_level']=cl['solution'].apply(tree_level_ii)

In [41]:
cl[:3]

,l,k,solution,gcd,n,tree_level
0,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,NaN
1,"[1, -1, 0, 1]","[-2, 0, 2, -2]","[1, 1, 1, -5, -6, 10, 10, 10, -11, -11]",1,10,"{'[ν,s]': [-11, 11], 'sp': None, 'sp_massless': None}"
2,"[1, -1, 0, 1]","[-2, -1, 2, -2]","[1, 1, 1, 4, 4, 5, -10, -10, -10, 14]",1,10,NaN


Remove solutions with `'sp_massless'` length > 0, i.e, with `'tree_level'=NaN`

In [42]:
cl=cl.dropna(subset=['tree_level']).reset_index(drop=True)
cl.shape

(8523, 6)

In [43]:
cl=cl[cl['n']<10].reset_index(drop=True)

In [44]:
cl.shape

(1038, 6)

In [45]:
cl[cl['tree_level'].apply(lambda d: d.get('sp') is None)].shape

(28, 6)

In [48]:
cl['ν']=cl['tree_level'].apply(lambda d: d.get('[ν,s]')[0]  )
cl['sp']=cl['tree_level'].apply(lambda d: d.get('sp') )
cl['sp']=cl['sp'].fillna('None').apply(lambda x: eval(x) if x==np.nan else x)
cl['massless']=cl.apply(lambda row: extract_Dirac_and_Majorana
                       (
                        list(np.unique([x for x in row['solution'] if x!=row['ν'] ])),-row.get('ν')
                                         ),axis='columns')
#Fix massless for rank zero

Reorder the filtered solutions

In [49]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl.sort_values(['n','smax']).reset_index(drop=True)
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,ν,sp,massless,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"{'[ν,s]': [1, -1], 'sp': -8, 'sp_massless': None}",1,-8,[],5


### Flag equivalent solutions

In [50]:
def characterize_solution(cl,i,j=0):
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    cl.loc[i,'trptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>2] )
    cl.loc[i,'frptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>3] )
    return cl

Example: massless solutions

In [51]:
for i in cl.index:
    cl=characterize_solution(cl,i,0)

In [52]:
cl['nsp']=cl['tree_level'].apply(lambda d: 0 if d.get('sp') is None else 1)

### Add solutions with massless fermions that get masses with some $S'$

In [53]:
for k  in ['nsp','rptd','trptd','frptd']:
    cl[k]=cl[k].astype(int)

In [54]:
cl=cl.sort_values(['n','smax','nsp','rptd','trptd','frptd']).reset_index(drop=True)

In [55]:
cl[cl['solution'].astype(str)=='[1, 1, -4, -5, 9, 9, 9, -10, -10]']

,l,k,solution,gcd,n,tree_level,ν,sp,massless,smax,rptd,trptd,frptd,nsp
398,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,"{'[ν,s]': [9, -9], 'sp': None, 'sp_massless': None}",9,None,[],10,3,1,0,0


## Build the final table dropping out equivalent solutions 

In [56]:
fl=pd.DataFrame()

In [57]:
fl=fl.append( cl[ cl['n']==6 ].drop_duplicates(subset=['nsp','rptd','trptd','frptd']).reset_index(drop=True) )

In [58]:
fl=fl.append(cl[ cl['n']==7 ].drop_duplicates(subset=['nsp','rptd','trptd','frptd']).reset_index(drop=True) )

In [59]:
fl=fl.append(cl[ cl['n']==8 ].drop_duplicates(subset=['nsp','rptd','trptd','frptd']).reset_index(drop=True) )

In [60]:
fl=fl.append( cl[ cl['n']==9 ].drop_duplicates(subset=['nsp','rptd','trptd','frptd']).reset_index(drop=True) )

In [61]:
fl=fl.reset_index(drop=True)

In [62]:
fl

,l,k,solution,gcd,n,tree_level,ν,sp,massless,smax,rptd,trptd,frptd,nsp
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"{'[ν,s]': [1, -1], 'sp': -8, 'sp_massless': None}",1,-8,[],5,2,1,0,1
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,"{'[ν,s]': [5, -5], 'sp': None, 'sp_massless': None}",5,None,[],6,1,0,0,0
2,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,6,"{'[ν,s]': [1, -1], 'sp': 3, 'sp_massless': []}",1,3,"[-11, 8]",17,1,0,0,1
3,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,7,"{'[ν,s]': [-6, 6], 'sp': 3, 'sp_massless': []}",-6,3,"[-4, 7]",7,1,0,0,1
4,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,7,"{'[ν,s]': [-9, 9], 'sp': 18, 'sp_massless': []}",-9,18,"[7, 11]",11,1,1,0,1
5,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,"{'[ν,s]': [2, -2], 'sp': 12, 'sp_massless': []}",2,12,[-6],7,1,1,0,1
6,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,8,"{'[ν,s]': [7, -7], 'sp': 14, 'sp_massless': []}",7,14,"[-8, -6]",8,1,0,0,1
7,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,8,"{'[ν,s]': [-10, 10], 'sp': 15, 'sp_massless': []}",-10,15,"[4, 7, 8, 11]",11,2,0,0,1
8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,8,"{'[ν,s]': [10, -10], 'sp': 5, 'sp_massless': []}",10,5,"[-7, 2]",11,3,0,0,1
9,"[-6, -3, -6, -2]","[-6, -4, -1, -5]","[1, 1, 2, 2, 3, -5, -6, -6, 8]",1,9,"{'[ν,s]': [2, -2], 'sp': -12, 'sp_massless': None}",2,-12,[],8,3,0,0,1


In [63]:
def getit(it,dk):
    it['ν']=dk.get('[ν,s]')[0]
    it['s']=dk.get('[ν,s]')[1]
    return it
    
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    k=fl.loc[i,'tree_level']
    dk=k
    it=getit(it,dk)
    tm=tm.append(it,ignore_index=True)

In [64]:
tm

,frptd,gcd,k,l,massless,n,nsp,rptd,s,smax,solution,sp,tree_level,trptd,ν
0,0.0,1.0,"[-1, 2]","[-1, -2]",[],6.0,1.0,2.0,-1.0,5.0,"[1, 1, 1, -4, -4, 5]",-8,"{'[ν,s]': [1, -1], 'sp': -8, 'sp_massless': None}",1.0,1.0
1,0.0,1.0,"[-2, 0]","[-1, 1]",[],6.0,0.0,1.0,-5.0,6.0,"[1, -2, -3, 5, 5, -6]",None,"{'[ν,s]': [5, -5], 'sp': None, 'sp_massless': None}",0.0,5.0
2,0.0,1.0,"[-1, -2, 1]","[0, 2]","[-11, 8]",6.0,1.0,1.0,-1.0,17.0,"[1, 1, 8, -11, -16, 17]",3,"{'[ν,s]': [1, -1], 'sp': 3, 'sp_massless': []}",0.0,1.0
3,0.0,1.0,"[-1, -2, -1]","[-1, 1]","[-4, 7]",7.0,1.0,1.0,6.0,7.0,"[1, 3, -4, 5, -6, -6, 7]",3,"{'[ν,s]': [-6, 6], 'sp': 3, 'sp_massless': []}",0.0,-6.0
4,0.0,1.0,"[-1, 0, -1]","[-1, 0, -2]","[7, 11]",7.0,1.0,1.0,9.0,11.0,"[1, 7, 8, -9, -9, -9, 11]",18,"{'[ν,s]': [-9, 9], 'sp': 18, 'sp_massless': []}",1.0,-9.0
5,0.0,1.0,"[-7, 4, 0]","[-1, 2, -2]",[-6],8.0,1.0,1.0,-2.0,7.0,"[1, 2, 2, 2, -3, -5, -6, 7]",12,"{'[ν,s]': [2, -2], 'sp': 12, 'sp_massless': []}",1.0,2.0
6,0.0,1.0,"[-7, -10, -4]","[3, 2, -2]","[-8, -6]",8.0,1.0,1.0,-7.0,8.0,"[2, -3, -4, 5, -6, 7, 7, -8]",14,"{'[ν,s]': [7, -7], 'sp': 14, 'sp_massless': []}",0.0,7.0
7,0.0,1.0,"[-1, 0, -3, 5]","[1, -2, 2]","[4, 7, 8, 11]",8.0,1.0,2.0,10.0,11.0,"[4, -5, -5, 7, 8, -10, -10, 11]",15,"{'[ν,s]': [-10, 10], 'sp': 15, 'sp_massless': []}",0.0,-10.0
8,0.0,2.0,"[-2, -5, 1]","[-2, -1, 2]","[-7, 2]",8.0,1.0,3.0,-10.0,11.0,"[1, 2, 2, -7, -7, 10, 10, -11]",5,"{'[ν,s]': [10, -10], 'sp': 5, 'sp_massless': []}",0.0,10.0
9,0.0,1.0,"[-6, -4, -1, -5]","[-6, -3, -6, -2]",[],9.0,1.0,3.0,-2.0,8.0,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",-12,"{'[ν,s]': [2, -2], 'sp': -12, 'sp_massless': None}",0.0,2.0


In [65]:
for k in ['n','gcd','ν','s','rptd','trptd']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs( row['solution'] ).max(),axis='columns' )
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
#tm['Q']=True
#sltns={'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],}
#for k in sltns.keys():
#    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
#          )].index[0] #unstable dark matter
#    tm.loc[iq,'Q']=False

#tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['n','l','k','solution','gcd','ν','s','massless','rptd','trptd']
  ].sort_values(['n','rptd','trptd'])#.loc[[0,4]]#.loc[[0,4]]

,n,l,k,solution,gcd,ν,s,massless,rptd,trptd
1,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,[],1,0
2,6,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,1,-1,"[-11, 8]",1,0
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-1,[],2,1
3,7,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,-6,6,"[-4, 7]",1,0
4,7,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,-9,9,"[7, 11]",1,1
6,8,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,7,-7,"[-8, -6]",1,0
5,8,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,2,-2,[-6],1,1
7,8,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,-10,10,"[4, 7, 8, 11]",2,0
8,8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,10,-10,"[-7, 2]",3,0
11,9,"[-3, -1, 5]","[-9, 3, -4, -1]","[1, 2, -3, 4, -5, -6, 8, 8, -9]",1,8,-8,"[-6, 2]",1,0


In [36]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp','rptd','trptd']].reset_index(drop=True)
kk['sltn']=kk['solution'].astype(str)
kk.sort_values('n').drop('sltn',axis='columns').reset_index(drop=True)

,n,l,k,solution,gcd,ν,s,massless,sp,rptd,trptd
0,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,[],None,1,0
1,6,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,1,-1,"[-11, 8]",3,1,0
2,7,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,-6,6,"[-4, 7]",3,1,0
3,7,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,-9,9,"[7, 11]",18,1,1
4,8,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,2,-2,[-6],12,1,1
5,8,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,7,-7,"[-8, -6]",14,1,0
6,8,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,-10,10,"[4, 7, 8, 11]",15,2,0
7,8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,10,-10,"[-7, 2]",5,3,0
8,9,"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,-7,7,[],None,1,1
9,9,"[-3, -1, 5]","[-9, 3, -4, -1]","[1, 2, -3, 4, -5, -6, 8, 8, -9]",1,8,-8,"[-6, 2]",4,1,0


In [37]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp']].copy()

In [780]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [781]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

0
8
11


In [782]:
cat solutions.tex

\begin{tabular}{llllrrrll}
\toprule
              n &                l &                 k &                           solution & gcd &   ν &   s &       massless &    sp \\
\midrule
 \boldsymbol{6} &          (-1, 1) &           (-2, 0) &              (1, -2, -3, 5, 5, -6) &   1 &   5 &  -5 &             () &  None \\
              6 &           (0, 2) &       (-1, -2, 1) &            (1, 1, 8, -11, -16, 17) &   1 &   1 &  -1 &       (-11, 8) &     3 \\
              7 &          (-1, 1) &      (-1, -2, -1) &           (1, 3, -4, 5, -6, -6, 7) &   1 &  -6 &   6 &        (-4, 7) &     3 \\
              7 &      (-1, 0, -2) &       (-1, 0, -1) &          (1, 7, 8, -9, -9, -9, 11) &   1 &  -9 &   9 &        (7, 11) &    18 \\
              8 &      (-1, 2, -2) &        (-7, 4, 0) &        (1, 2, 2, 2, -3, -5, -6, 7) &   1 &   2 &  -2 &           (-6) &    12 \\
              8 &       (3, 2, -2) &     (-7, -10, -4) &       (2, -3, -4, 5, -6, 7, 7, -8) &   1 &   7 &  -7 &       (-8, -6) 

In [289]:
kkk

,n,l,k,solution,gcd,ν,s,massless
0,\boldsymbol{6},"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,0.0
1,\boldsymbol{9},"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,-7,7,0.0
2,\boldsymbol{9},"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,-9,0.0
